In [89]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import bisplrep, bisplev
from scipy import integrate

In [116]:
def create_knots(k,n,min=0,max=10):
    # Create open-uniform knot vector for a B-spline of degree k and with n+1 control points between the min and max values.  
    return np.concatenate(
        [(min)*np.zeros(k),
         np.linspace(0,max,n),
         (max)*np.ones(k)]
        )    

def fit_Bspline(x,y,z,k,tx,ty):
    #Fit B-spline on the given dataset of x,y,z and return its tck form and the depth-error.
    tck,depth_loss,_,_=bisplrep(x, y, z, kx=k, ky=k, task=-1, s=0, tx=tx, ty=ty, full_output=1)
    return tck, depth_loss

def curvature_loss(tck):
    
    

In [118]:
#This is just an example.

#Generate random data points
np.random.seed(42)  # For reproducibility
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
z = np.random.random((100, 100))

# Create a meshgrid from x and y
X, Y = np.meshgrid(x, y)

# Flatten the meshgrid arrays and z array to pass to bisplrep
x_flat = X.flatten()
y_flat = Y.flatten()
z_flat = z.flatten()

#Define the parameters
k =3 #Degree (both x and y direction)
n = 10 # The number of control points (n+1) 

tx=create_knots(k,n)
ty=create_knots(k,n)

#Create a B-Spline representation using previous determined knots on the data and return the error. 
tck,_=fit_Bspline(x_flat, y_flat, z_flat, k=k, tx=tx, ty=ty)